In [1]:
#| default_exp cfg
from nbdev import *
from nbdev.showdoc import *

# Earley Parser

> Parses it earley style

In [2]:
#| exporti
from pprint import pprint
from typing import NamedTuple
from functools import cache

In [3]:
class State(object):
    def __init__(self, label, rules, dot_idx, start_idx, end_idx, idx, made_from, producer):
        self.label = label
        self.rules = rules
        self.dot_idx = dot_idx
        self.start_idx = start_idx
        self.end_idx = end_idx
        self.idx = idx
        self.made_from = made_from
        self.producer = producer

    def next(self):
        """Returns the tag after the dot"""
        return self.rules[self.dot_idx]

    def complete(self):
        return len(self.rules) == self.dot_idx

    def __eq__(self, other):
        return (self.label == other.label and
                self.rules == other.rules and
                self.dot_idx == other.dot_idx and
                self.start_idx == other.start_idx and
                self.end_idx == other.end_idx)

    def __str__(self):
        rule_string = ''
        for i, rule in enumerate(self.rules):
            if i == self.dot_idx:
                rule_string += '\\bullet '
            rule_string += rule + ' '
        if self.dot_idx == len(self.rules):
            rule_string += '\\bullet'
        return 'S%d %s -> %s [%d, %d] %s %s' % (self.idx, self.label, rule_string, self.start_idx, 
                                                self.end_idx, self.made_from, self.producer)

class Earley:
    def __init__(self, words, grammar, terminals):
        self.chart = [[] for _ in range(len(words) + 1)]
        self.current_id = 0
        self.words = words
        self.grammar = grammar
        self.terminals = terminals

    def get_new_id(self):
        self.current_id += 1
        return self.current_id - 1

    def is_terminal(self, tag):
        return tag in self.terminals

    def is_complete(self, state):
        return len(state.rules) == state.dot_idx

    def enqueue(self, state, chart_entry):
        if state not in self.chart[chart_entry]:
            self.chart[chart_entry].append(state)
        else:
            self.current_id -= 1

    def predictor(self, state):
        for production in self.grammar[state.next()]:
            self.enqueue(State(state.next(), production, 0, state.end_idx, state.end_idx, self.get_new_id(), [], 'predictor'), state.end_idx)

    def scanner(self, state):
        if self.words[state.end_idx] in self.grammar[state.next()]:
            self.enqueue(State(state.next(), [self.words[state.end_idx]], 1, state.end_idx, state.end_idx + 1, self.get_new_id(), [], 'scanner'), state.end_idx + 1)

    def completer(self, state):
        for s in self.chart[state.start_idx]:
            if not s.complete() and s.next() == state.label and s.end_idx == state.start_idx and s.label != 'gamma':
                self.enqueue(State(s.label, s.rules, s.dot_idx + 1, s.start_idx, state.end_idx, self.get_new_id(), s.made_from + [state.idx], 'completer'), state.end_idx)

    def parse(self):
        self.enqueue(State('gamma', ['S'], 0, 0, 0, self.get_new_id(), [], 'dummy start state'), 0)
        
        for i in range(len(self.words) + 1):
            for state in self.chart[i]:
                if not state.complete() and not self.is_terminal(state.next()):
                    self.predictor(state)
                elif i != len(self.words) and not state.complete() and self.is_terminal(state.next()):
                    self.scanner(state)
                else:
                    self.completer(state)

    def __str__(self):
        res = ''
        
        for i, chart in enumerate(self.chart):
            res += '\nChart[%d]\n' % i
            for state in chart:
                res += str(state) + '\n'

        return res


def test():
    grammar = {
        'S':           [['NP', 'VP'], ['Aux', 'NP', 'VP'], ['VP']],
        'NP':          [['Det', 'Nominal'], ['Proper-Noun']],
        'Nominal':     [['Noun'], ['Noun', 'Nominal']],
        'VP':          [['Verb'], ['Verb', 'NP']],
        'Det':         ['that', 'this', 'a'],
        'Noun':        ['book', 'flight', 'meal', 'money'],
        'Verb':        ['book', 'include', 'prever'],
        'Aux':         ['does'],
        'Prep':        ['from', 'to', 'on'],
        'Proper-Noun': ['Houston', 'TWA']
    }
    terminals = ['Det', 'Noun', 'Verb', 'Aux', 'Prep', 'Proper-Noun']

    earley = Earley(['book', 'that', 'flight'], grammar, terminals)
    earley.parse()
    print(earley)
test()


Chart[0]
S0 gamma -> \bullet S  [0, 0] [] dummy start state
S1 S -> \bullet NP VP  [0, 0] [] predictor
S2 S -> \bullet Aux NP VP  [0, 0] [] predictor
S3 S -> \bullet VP  [0, 0] [] predictor
S4 NP -> \bullet Det Nominal  [0, 0] [] predictor
S5 NP -> \bullet Proper-Noun  [0, 0] [] predictor
S6 VP -> \bullet Verb  [0, 0] [] predictor
S7 VP -> \bullet Verb NP  [0, 0] [] predictor

Chart[1]
S8 Verb -> book \bullet [0, 1] [] scanner
S9 VP -> Verb \bullet [0, 1] [8] completer
S10 VP -> Verb \bullet NP  [0, 1] [8] completer
S11 S -> VP \bullet [0, 1] [9] completer
S12 NP -> \bullet Det Nominal  [1, 1] [] predictor
S13 NP -> \bullet Proper-Noun  [1, 1] [] predictor

Chart[2]
S14 Det -> that \bullet [1, 2] [] scanner
S15 NP -> Det \bullet Nominal  [1, 2] [14] completer
S16 Nominal -> \bullet Noun  [2, 2] [] predictor
S17 Nominal -> \bullet Noun Nominal  [2, 2] [] predictor

Chart[3]
S18 Noun -> flight \bullet [2, 3] [] scanner
S19 Nominal -> Noun \bullet [2, 3] [18] completer
S20 Nominal -> Nou

In [4]:
class State(NamedTuple):
    symbol: str
    rule: tuple
    startidx: int
    position: int
       
    def __repr__(self):
        pos = self.position
        return f"symbol {self.symbol:<4}: {str(self.rule[:pos]):<15} • {str(self.rule[pos:]):<30} {'idx('}{self.startidx}) {'Done' if self.is_complete() else ''})"
    
    def is_complete(self):
        return self.position == len(self.rule)
    
    def nextsym(self):
        return False if self.is_complete() else self.rule[self.position]

def parse(grammar, message, start):  
    def scan(column, parsedsym):
        # we are scanning the states in a column to check if we can advance based on the parsed symbol
        advancedstates = []
        for state in column:
            if not state.is_complete() and state.nextsym() == parsedsym:
                symbol, rule, startidx, position = state
                position += 1
                advancedstates.append(State(symbol, rule, startidx, position))
        # print('parsed symbol:', message[colidx-1], ':we have advances on', [s['symbol'] for s in advancedstates])
        return advancedstates

    @cache
    def predict(colidx, symbol):
        seen = {symbol} # otherwise you can have infinite loop when recursively adding
        toadd = {symbol}
        predicted = set()
        while toadd:
            cur = toadd.pop()
            for rule in grammar.get(cur, []):
                newstate = State(cur, rule, colidx, 0)
                predicted.add(newstate)
                if (recursivesymbol := newstate.rule[0]) not in seen:
                    toadd.add(recursivesymbol)
                    seen.add(recursivesymbol)
        return frozenset(predicted)
    
    def isvalid(cols):
        for state in cols[colidx]:
            if state.is_complete() and state.startidx == 0 and state.symbol == start:
                return True #('valid')
        else:
            return False

    def __init__():        
        cols = [set() for _ in range(len(message)+1)]
        cols[0] |= predict(0, start)
        pprint(cols[0])
        return cols
    
    cols = __init__()
    for colidx in range(1, len(message)+1): # we just populated the 0th col, now the first character has colidx 1
        print(colidx, message[colidx-1], len(cols[colidx-1]))
        advancedstates = scan(cols[colidx-1], message[colidx-1])
        # check if any of the states are completed, if yes do scan again (loop untill nothing is completed)
        while advancedstates:
            advancedstate = advancedstates.pop()
            cols[colidx].add(advancedstate)
            if advancedstate.is_complete(): # do scan again, looking to advance other states based on the symbol of the completed rule
                # print('completion:', advancedstate['symbol'])
                advancedstates += scan(cols[advancedstate.startidx], advancedstate.symbol)
            else: # predict new states based on the next expected symbol in the rule
                cols[colidx] |= predict(colidx, advancedstate.nextsym())
    return isvalid(cols)

In [5]:
# https://adventofcode.com/2020/day/19
rules, messages = open('cfg.txt').read().split('\n\n')
grammar = {}

for rule in rules.split('\n'):
    num, makefrom = rule.split(': ')
    makefrom = makefrom.replace('"', '')
    makefrom = tuple(makefrom.split(' | '))
    makefrom = tuple(tuple(option.split()) for option in makefrom)
    grammar[num] = makefrom
    
messages = messages.split('\n')
s = '0'
ans = 0
for m in messages:
    ans += parse(grammar, m,s)
ans

{symbol 0   : ()              • ('8', '11')                    idx(0) ),
 symbol 108 : ()              • ('57', '95')                   idx(0) ),
 symbol 108 : ()              • ('83', '83')                   idx(0) ),
 symbol 131 : ()              • ('20', '83')                   idx(0) ),
 symbol 131 : ()              • ('74', '57')                   idx(0) ),
 symbol 133 : ()              • ('57', '79')                   idx(0) ),
 symbol 133 : ()              • ('83', '13')                   idx(0) ),
 symbol 14  : ()              • ('57', '124')                  idx(0) ),
 symbol 14  : ()              • ('83', '138')                  idx(0) ),
 symbol 15  : ()              • ('35', '83')                   idx(0) ),
 symbol 15  : ()              • ('53', '57')                   idx(0) ),
 symbol 20  : ()              • ('57', '127')                  idx(0) ),
 symbol 20  : ()              • ('83', '80')                   idx(0) ),
 symbol 30  : ()              • ('57', '70')       

129